<a href="https://colab.research.google.com/github/Imanchandra2024/FeyynMcDonalds/blob/main/fyenn_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np


data = {
    'Anna': [0, 0],
    'Bill': [0, 0],
    'Frank': [40, 40],
    'Julia': [30, 30],
    'Maria': [20, 20],
    'Michael': [95, 95],
    'Tom': [43.64, 43.64]
}
df = pd.DataFrame(data).T
df.columns = ['X', 'Y']


dist_matrix = pdist(df.values, metric='euclidean')
square_dist = squareform(dist_matrix)

# Round and convert to DataFrame for pretty output
names = df.index.tolist()
rounded_df = pd.DataFrame(np.round(square_dist, 2), index=names, columns=names)

print(rounded_df)



           Anna    Bill  Frank  Julia   Maria  Michael    Tom
Anna       0.00    0.00  56.57  42.43   28.28   134.35  61.72
Bill       0.00    0.00  56.57  42.43   28.28   134.35  61.72
Frank     56.57   56.57   0.00  14.14   28.28    77.78   5.15
Julia     42.43   42.43  14.14   0.00   14.14    91.92  19.29
Maria     28.28   28.28  28.28  14.14    0.00   106.07  33.43
Michael  134.35  134.35  77.78  91.92  106.07     0.00  72.63
Tom       61.72   61.72   5.15  19.29   33.43    72.63   0.00


In [6]:
import pandas as pd
from sklearn.metrics import pairwise_distances
import numpy as np

# Simulated numeric version of 'annabill' data
data = {
    'Anna': [0, 0],
    'Bill': [0, 0],
    'Frank': [40, 40],
    'Julia': [30, 30],
    'Maria': [20, 20],
    'Michael': [95, 95],
    'Tom': [43.64, 43.64]
}
df = pd.DataFrame(data).T
df.columns = ['X', 'Y']  # Give coordinate column names

# Compute Euclidean distances using scikit-learn (same as daisy with metric='euclidean')
dissimilarities = pairwise_distances(df, metric='euclidean')
dissim_df = pd.DataFrame(np.round(dissimilarities, 2), index=df.index, columns=df.index)

print("Dissimilarities (Euclidean):")
print(dissim_df)


Dissimilarities (Euclidean):
           Anna    Bill  Frank  Julia   Maria  Michael    Tom
Anna       0.00    0.00  56.57  42.43   28.28   134.35  61.72
Bill       0.00    0.00  56.57  42.43   28.28   134.35  61.72
Frank     56.57   56.57   0.00  14.14   28.28    77.78   5.15
Julia     42.43   42.43  14.14   0.00   14.14    91.92  19.29
Maria     28.28   28.28  28.28  14.14    0.00   106.07  33.43
Michael  134.35  134.35  77.78  91.92  106.07     0.00  72.63
Tom       61.72   61.72   5.15  19.29   33.43    72.63   0.00
